In [1]:
# %pip install mlxtend

In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split

C:\Users\austinsh\AppData\Local\Temp\ipykernel_60288\2355519372.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [5]:
# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

# print(df_All.head())

In [6]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Octanol', 'Hexanol', 'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [7]:
# selected_columns = ['425 %Al', 'Butanol', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
#        'FC55003', 'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555',
#        'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
#        'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TC55553', 'TC55555'
#                    ]

# existing_columns = [col for col in selected_columns if col in df_All.columns]
# df_All = df_All[existing_columns]

In [8]:
df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\3 Feature Selection\filtered-out_5-9_corr.csv')

print(df_All.head())

                  Date  425_pct_Al     Al2O3  M_Value  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-17 09:00:00    6.319560  11.41670  3.50773    2.554580     22.0531   
1  2012-05-17 10:00:00    6.319915  11.40835  3.50797    2.555935     22.0557   
2  2012-05-17 12:00:00    6.320970  11.39165  3.50869    2.560005     22.0636   
3  2012-05-18 08:00:00    6.328690  11.39165  3.51401    2.540370     22.0604   
4  2012-05-18 09:00:00    6.336060  11.40500  3.51910    2.514430     22.0484   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O  Decanol   DI55102   DI55152  \
0            0.867508    0.670721     3.354160  1.03026  0.965043  0.924515   
1            0.864729    0.671059     3.306475  1.03026  0.965043  0.924515   
2            0.856391    0.672071     3.163420  1.06326  0.965043  0.924515   
3            0.811652    0.674174     2.238130  1.51850  0.971625  0.933431   
4            0.773793    0.675408     1.391475  1.51850  0.971625  0.933431   

    DI55580   FC55003    FC55009   FC5

In [9]:
df_All.columns

Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Decanol', 'DI55102',
       'DI55152', 'DI55580', 'FC55003', 'FC55009', 'FC55552', 'FC55569',
       'FC55576', 'FFC55553', 'FFC55555', 'LC52572', 'LC55553', 'LC55557',
       'LC55568', 'LC90366', 'LC90368', 'PI55004', 'PI55020', 'TC55552',
       'TC55555', 'TI40050', 'TI55021'],
      dtype='object')

In [10]:
# Splitting into train and test
X = df_All.drop(['Decanol', 'Date'], axis=1)  # Assuming 'target' is your target column
y = df_All['Decanol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a linear regression model
linreg = LinearRegression()

# Create a SequentialFeatureSelector object
sfs = SFS(linreg, 
          k_features='best', 
          forward=True, 
          floating=True, 
          scoring='r2',
          cv=5)

# Fit the model
sfs = sfs.fit(X_train, y_train)

# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]

# Build the final model using selected features
final_model = linreg.fit(X_train[selected_features], y_train)

# Optionally, evaluate the model on the test set
y_pred = final_model.predict(X_test[selected_features])

c:\Users\austinsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\austinsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\austinsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\austinsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warning

In [12]:
# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
print("Selected Features:")
print(selected_features)

# Display the coefficients of the final model
print("\nModel Coefficients:")
for i, feature in enumerate(selected_features):
    print(f"{feature}: {final_model.coef_[i]}")

# Display the R-squared value for the training set
r_squared_train = final_model.score(X_train[selected_features], y_train)
print(f"\nR-squared on Training Set: {r_squared_train}")

# Optionally, if you evaluated the model on a test set
r_squared_test = final_model.score(X_test[selected_features], y_test)
print(f"R-squared on Test Set: {r_squared_test}")


Selected Features:
Index(['425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'DI55102', 'DI55152',
       'FC55003', 'FC55009', 'FC55552', 'FC55569', 'FC55576', 'FFC55553',
       'FFC55555', 'LC52572', 'LC55553', 'LC55568', 'LC90366', 'LC90368',
       'PI55004', 'PI55020', 'TC55552', 'TC55555', 'TI40050', 'TI55021'],
      dtype='object')

Model Coefficients:
425_pct_Al: -0.1283522804950389
Al2O3: -0.11867476568376022
M_Value: -1.44874498116115
C4_pct_Eth: -0.18283520179742294
C4_pct_H2O: 0.022335882864707553
HydWtr_pct_Ammonia: -0.47729696170431724
C4_pct_Hex: 0.2932345190838613
HydWtr_Na2O: -0.044750135404388505
DI55102: 0.5265336153331183
DI55152: 4.155042765382263
FC55003: -0.00011361865319652907
FC55009: -0.0009492407288831164
FC55552: 6.523046168447393e-05
FC55569: -0.00019602026859377467
FC55576: -0.00010143900971451504
FFC55553: -6.659496001852079
FFC55555: 1.8319427688503884
LC52572: 0.018485854612332507
LC

In [13]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-03-21 03:28:01.383072
